In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold

random_seed = 42
np.random.seed(random_seed)

df = pd.read_csv("heart_failure_clinical_records_dataset.csv")
data = df.to_numpy()
np.random.shuffle(data)
training_data = data[np.random.choice(data.shape[0], 100, replace=False)]
labels_array = training_data[:, -1]
labels_array = labels_array.astype(int)
features = training_data[:, :-1]



dense1 = Layer_Dense(12, 64, weight_regularizer_l2=0.0009,
bias_regularizer_l2=0.0009)
activation1 = Activation_ReLU()
dense2 = Layer_Dense(64, 2)
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = Optimizer_Adam(learning_rate=0.001, decay=5e-4)
# Train in loop
for epoch in range(100001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(features)
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    data_loss = loss_activation.forward(dense2.output, labels_array)
    # Calculate regularization penalty
    regularization_loss = \
    loss_activation.loss.regularization_loss(dense1) + \
    loss_activation.loss.regularization_loss(dense2)
    # Calculate overall loss
    loss = data_loss + regularization_loss
    # Calculate accuracy from output of activation2 and targets # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(labels_array.shape) == 2:
        labels_array = np.argmax(labels_array, axis=1) 
    accuracy = np.mean(predictions==labels_array)

    # Backward pass
    loss_activation.backward(loss_activation.output, labels_array)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    # Update weights and biases
    optimizer.pre_update_params()

    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()


testing_data = data[np.random.choice(data.shape[0], 100, replace=False)]
testing_labels = training_data[:, -1]
testing_labels = testing_labels.astype(int)
testing_features = training_data[:, :-1]
dense1.forward(testing_features)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
loss = loss_activation.forward(dense2.output, testing_labels)
predictions = np.argmax(loss_activation.output, axis=1)
if len(testing_labels.shape) == 2:
    testing_labels = np.argmax(testing_labels, axis=1) 
accuracy = np.mean(predictions == testing_labels)
print(f'validation, acc: {accuracy:.3f}, loss: {loss:.3f}')

validation, acc: 0.930, loss: 0.208
